In [52]:
import pandas as pd

In [53]:
# open files
data = pd.read_csv('final_data.csv')
variables = pd.read_csv(r'variables/variables_acs5_2022.csv')
# if the last letter is E, remove it, and if the last letter is P, remove that too in Name
variables['Name'] = variables['Name'].apply(lambda x: x[:-2] if x[-2] == 'P' else x[:-1])
# open missing_data.csv
missing_data = pd.read_csv('missing_data.csv')[['variable', 'prop_missing']]
# calculate prop_available
missing_data['prop_available'] = 1 - missing_data['prop_missing']
missing_data = missing_data.drop(columns='prop_missing')
missing_data = missing_data.groupby('variable').mean().reset_index()

In [54]:
meta_data = {'Variable': [], 'Label': [], 'Concept': [], 'Available Elections': [], 'Missing Elections': [], 'Percent Elections Available': [], 'Average Tract Availability': []}

for column in data.columns[1:]:
    if column.endswith('PE'):
        stem = column[:-2]
    else:
        stem = column[:-1]
    label = variables[variables['Name'] == stem]['Label'].values[0]
    concept = variables[variables['Name'] == stem]['Concept'].values[0]

    # get the number of rows with null values
    null_rows = data[column].isnull().sum()
    # get the number of total rows
    total_rows = data.shape[0]
    present_rows = total_rows - null_rows
    # get the percentage of present values
    present_percentage = present_rows / total_rows
    # get prop_available if variable is there, otherwise it is 1
    tract_availability = 1 if column not in missing_data['variable'].values else missing_data[missing_data['variable'] == column]['prop_available'].values[0]

    meta_data['Variable'].append(column)
    meta_data['Label'].append(label)
    meta_data['Concept'].append(concept)
    meta_data['Available Elections'].append(present_rows)
    meta_data['Missing Elections'].append(null_rows)
    meta_data['Percent Elections Available'].append(present_percentage)
    meta_data['Average Tract Availability'].append(tract_availability)

meta_data = pd.DataFrame(meta_data)
meta_data.to_csv('meta_data.csv', index=False)

IndexError: index 0 is out of bounds for axis 0 with size 0